In [1]:
from pathlib import Path
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
TRAINING_DATA = Path("./data/training_data/")

In [16]:
openai.File.create(
    file = open(TRAINING_DATA/"training_data.jsonl", "rb"),
    purpose = "fine-tune"
)

<File file id=file-L9O7WT2SPWlNiRv8XOaNx2xh at 0x144ece1b0> JSON: {
  "object": "file",
  "id": "file-L9O7WT2SPWlNiRv8XOaNx2xh",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 1506517,
  "created_at": 1696589710,
  "status": "uploaded",
  "status_details": null
}

In [17]:
openai.FineTuningJob.create(training_file = "file-L9O7WT2SPWlNiRv8XOaNx2xh", model = "gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-QOdnk3QxBVDBHM1Lxwk3155U at 0x14344f290> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-QOdnk3QxBVDBHM1Lxwk3155U",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1696589729,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-I4j7CvqT1sUoMLq85ywq27zV",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-L9O7WT2SPWlNiRv8XOaNx2xh",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [28]:
2%2

0

In [26]:
openai.FineTuningJob.retrieve("ftjob-QOdnk3QxBVDBHM1Lxwk3155U")

<FineTuningJob fine_tuning.job id=ftjob-QOdnk3QxBVDBHM1Lxwk3155U at 0x161b32a20> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-QOdnk3QxBVDBHM1Lxwk3155U",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1696589729,
  "finished_at": 1696593019,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::86ddUkes",
  "organization_id": "org-I4j7CvqT1sUoMLq85ywq27zV",
  "result_files": [
    "file-M7IyM6lqF4QZDSXx0RDF9lkz"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-L9O7WT2SPWlNiRv8XOaNx2xh",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 769686,
  "error": null
}